In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import pickle

from label_prop import PropagateLabels

In [2]:
# Load Graph.
G = nx.read_gexf("../../state_files/PyPi Network V4.gexf")
# Keep only giant component.
gc_nodes = sorted(nx.connected_components(G.to_undirected()), key = lambda x: len(x), reverse=True)[0]
not_gc_nodes = set(G.nodes()) - gc_nodes
G.remove_nodes_from(not_gc_nodes)

# Load labels.
with open("../../state_files/PyPi Dataframe V4.pickle", "rb") as f:
    df = pickle.load(f)
labels = df["Topic"].dropna().to_dict()
# Remove 'Other' topic.
for n, l in labels.items():
    if 'Other/Nonlisted Topic' in l:
        l.remove('Other/Nonlisted Topic')
# Filter nodes with no topics.
labels = dict(filter(lambda x: len(x[1]) > 0, labels.items()))
# Keep nodes from giant component.
labels = dict(filter(lambda x: x[0] in gc_nodes, labels.items()))

In [3]:
pl = PropagateLabels(G, labels)
final_labels = pl.propagate_all()

100%|██████████| 93791/93791 [14:28<00:00, 108.03it/s] 


154163 nodos etiquetados de 183495.


100%|██████████| 7787/7787 [02:40<00:00, 48.60it/s] 


161950 nodos etiquetados de 183495.


100%|██████████| 1057/1057 [00:29<00:00, 36.06it/s]


...

163208 nodos etiquetados de 183495.


100%|██████████| 1/1 [00:00<00:00, 52.69it/s]


163209 nodos etiquetados de 183495.


100%|██████████| 163209/163209 [00:00<00:00, 1460705.96it/s]
0it [00:00, ?it/s]

163209 nodos etiquetados de 183495.
No se encontraron nuevas etiquetas.


In [5]:
len(final_labels)

163209